## 3 Course 1.1.6 Practical Activity: Conduct A/B testing in Python

An online bicycle store has changed its home page interface to encourage visitors to click through to their loyalty programme sign-up page. They are hoping the new interface will encourage more visitors to access the loyalty programme page, see what benefits the programme brings, and sign up. The current click-through rate sits at around 50% annually, and the company hopes the new design will push this to at least 55%.

The company has asked you to run an A/B test to calculate whether the change to the home page interface has made a difference and whether more visitors have clicked through to the loyalty programme page. They’ve provided the relevant data set as a CSV file, and they have told you that users on the server ID1 are the treatment group and those on ID2 and ID3 are the control group. 

In [30]:
# Import the libraries, packages and classes
import pandas as pd
import math
import numpy as np
import statsmodels.stats.api as sms
import scipy.stats as st
import matplotlib as mpl
import matplotlib.pyplot as plot

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
# import the necessary libraries
from statsmodels.stats.power import TTestIndPower

### Work out the sample size

In [33]:
# Specify the three required parameters for the power analysis:
alpha = 0.05
power = 0.8
effect = sms.proportion_effectsize(0.50, 0.55)

# Perform power analysis by using the solve_power() function:
# Specify an instance of 'TTestIndPower'.
analysis = TTestIndPower()
#Calculate the sample size and list the parameters
result = analysis.solve_power(effect, power=power, nobs1=None, ratio=1.0, alpha=alpha)

# Print the output.
print('Sample Size: %.3f' % result)

Sample Size: 1565.490


### Load the data

In [34]:
# Read the CSV file
df = pd.read_csv('new_bike_shop_AB.csv')

# View the DataFrame
df.head()

,Unnamed: 0,RecordID,IPAddress,LoyaltyPage,ServerID,VisitPageFlag
0,0,1,39.13.114.2,1,2,0
1,1,2,13.3.25.8,1,1,0
2,2,3,247.8.211.8,1,1,0
3,3,4,124.8.220.3,0,3,0
4,4,5,60.10.192.7,0,2,0


In [35]:
df.shape

(184588, 6)

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184588 entries, 0 to 184587
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   Unnamed: 0     184588 non-null  int64 
 1   RecordID       184588 non-null  int64 
 2   IPAddress      184588 non-null  object
 3   LoyaltyPage    184588 non-null  int64 
 4   ServerID       184588 non-null  int64 
 5   VisitPageFlag  184588 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 8.4+ MB


### Clean the data set

In [37]:
# Check for duplicates
print(df[df.IPAddress.duplicated()])

        Unnamed: 0  RecordID     IPAddress  LoyaltyPage  ServerID  \
275            275       276    191.4.97.7            0         2   
394            394       395     79.9.70.7            1         3   
703            703       704    175.1.81.8            1         3   
809            809       810    125.0.30.9            1         2   
889            889       890  207.14.157.6            1         3   
...            ...       ...           ...          ...       ...   
184582      184582    184583    90.4.224.4            0         3   
184583      184583    184584   114.8.104.1            0         1   
184585      184585    184586   170.13.31.9            0         2   
184586      184586    184587   195.14.92.3            0         3   
184587      184587    184588  172.12.115.8            0         2   

        VisitPageFlag  
275                 0  
394                 0  
703                 0  
809                 0  
889                 0  
...               ...  
184

In [46]:
# Drop the duplicates by IPAddress
df2 = df.drop_duplicates(subset = 'IPAddress')

In [47]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39608 entries, 7 to 184584
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     39608 non-null  int64 
 1   RecordID       39608 non-null  int64 
 2   IPAddress      39608 non-null  object
 3   LoyaltyPage    39608 non-null  int64 
 4   ServerID       39608 non-null  int64 
 5   VisitPageFlag  39608 non-null  int64 
dtypes: int64(5), object(1)
memory usage: 2.1+ MB


### Remove unnecessary columns

In [48]:
# Drop irrelevant columns: Unnamed, RecordID and VisitPageFlag
# Specify that Unnamed, RecordID and VisitPageFlag are columns (i.e. axis=1)
df_final = df2.drop(['Unnamed: 0', 'RecordID', 'VisitPageFlag'], axis=1)

# Check the DataFrame
df_final.head()

,IPAddress,LoyaltyPage,ServerID
7,97.6.126.6,0,3
12,188.13.62.2,0,3
14,234.1.239.1,0,2
15,167.15.157.7,0,2
16,123.12.229.8,0,1


In [49]:
df_final.shape

(39608, 3)

In [50]:
df_final.info

<bound method DataFrame.info of            IPAddress  LoyaltyPage  ServerID
7         97.6.126.6            0         3
12       188.13.62.2            0         3
14       234.1.239.1            0         2
15      167.15.157.7            0         2
16      123.12.229.8            0         1
...              ...          ...       ...
184570     11.2.87.5            0         3
184573     18.6.86.9            0         1
184578  199.14.104.7            1         2
184580    93.3.115.6            1         3
184584   207.2.110.5            0         2

[39608 rows x 3 columns]>

### Subset the DataFrame with map() method

In [51]:
# Split data set into ID1 as treatment and ID2 & ID3 as control groups.
df_final['Group'] = df_final['ServerID'].map({1:'Treatment', 2:'Control', 3:'Control'})

# View DataFrame.
print(df_final.shape)
df_final.head()

(39608, 4)


,IPAddress,LoyaltyPage,ServerID,Group
7,97.6.126.6,0,3,Control
12,188.13.62.2,0,3,Control
14,234.1.239.1,0,2,Control
15,167.15.157.7,0,2,Control
16,123.12.229.8,0,1,Treatment


In [52]:
# count the values
df_final["Group"].value_counts()

Control      26310
Treatment    13298
Name: Group, dtype: int64

### Create the Sample: Split the df into two: control and treatment

In [56]:
# Create two DataFrames.
# You can use any random_state.
c_sample = df_final[df_final['Group'] == 'Control'].sample(n=1566,
                                                           random_state=22) 

t_sample = df_final[df_final['Group'] == 'Treatment'].sample(n=1566,
                                                             random_state=22)

# View the DataFrames.
print(c_sample)
print(t_sample)

           IPAddress  LoyaltyPage  ServerID    Group
178875   110.13.32.9            0         3  Control
127932     11.1.11.1            1         2  Control
20425    234.4.250.3            0         2  Control
104611   244.12.33.1            1         2  Control
132873    232.4.61.9            0         2  Control
...              ...          ...       ...      ...
118109   251.0.237.2            1         3  Control
87173     192.2.62.4            1         3  Control
44491     65.12.18.6            1         2  Control
65759   210.16.220.4            1         3  Control
174048   90.11.120.7            0         3  Control

[1566 rows x 4 columns]
          IPAddress  LoyaltyPage  ServerID      Group
127674   79.7.253.6            1         1  Treatment
36272     13.16.6.9            1         1  Treatment
100570  213.10.22.5            0         1  Treatment
162988   232.4.29.4            1         1  Treatment
159937   75.1.232.4            0         1  Treatment
...            

### Perform the A/B Test

In [57]:
# Join the two samples
ab_test = pd.concat([c_sample, t_sample], axis=0)

In [58]:
# Reset the A/B index
ab_test.reset_index(drop=True, inplace=True)

In [59]:
# View output.
ab_test

,IPAddress,LoyaltyPage,ServerID,Group
0,110.13.32.9,0,3,Control
1,11.1.11.1,1,2,Control
2,234.4.250.3,0,2,Control
3,244.12.33.1,1,2,Control
4,232.4.61.9,0,2,Control
...,...,...,...,...
3127,158.5.109.2,1,1,Treatment
3128,28.2.49.7,0,1,Treatment
3129,81.8.204.9,1,1,Treatment
3130,146.5.252.2,0,1,Treatment


In [60]:
# Import library
from scipy.stats import sem

In [70]:
# Calculate the conversion rates.
conversion_rates = ab_test.groupby('Group')['LoyaltyPage']


# Standard deviation of the proportion.
STD_p = lambda x: np.std(x, ddof=0)    
# Standard error of the proportion.
SE_p = lambda x: st.sem(x, ddof=0)     

conversion_rates = conversion_rates.agg([np.mean, STD_p, SE_p])
conversion_rates.columns = ['conversion_rate', 'std_deviation', 'std_error']


conversion_rates.style.format('{:.3f}')

,conversion_rate,std_deviation,std_error
Group,,,
Control,0.513,0.500,0.013
Treatment,0.498,0.500,0.013


### Calculate statistical significance

In [63]:
# Import proportions_ztest and proportion_confint from statsmodels
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [71]:
# Create a subset of control and treatment results
control_results = ab_test[ab_test['Group'] == 'Control']['LoyaltyPage']
treatment_results = ab_test[ab_test['Group'] == 'Treatment']['LoyaltyPage']

In [72]:
# Determine the count of the control_results and
# treatment_results sub-datasets and store them in their respective variables
n_con = control_results.count()
n_treat = treatment_results.count()

In [73]:
# Create a variable 'success' with the sum of the two data sets
# in a list format
success = [control_results.sum(), treatment_results.sum()]

In [74]:
# Create a variable 'nobs' which stores the values of
# variables n_con and n_treat in list format
nobs = [n_con, n_treat]

In [75]:
# Use the imported libraries to calculate the statistical values
z_stat, pval = proportions_ztest(success, nobs=nobs)
(lower_con, lower_treat), (upper_con, upper_treat) = proportion_confint(success, nobs=nobs, alpha=0.05)

In [76]:
# Print the outputs (with lead-in text)
print(f'Z test stat: {z_stat:.2f}')
print(f'P-value: {pval:.3f}')
print(f'Confidence Interval of 95% for control group: [{lower_con:.3f}, {upper_con:.3f}]')
print(f'Confidence Interval of 95% for treatment group: [{lower_treat:.3f}, {upper_treat:.3f}]')

Z test stat: 0.82
P-value: 0.411
Confidence Interval of 95% for control group: [0.488, 0.538]
Confidence Interval of 95% for treatment group: [0.473, 0.523]


### Summary Findings

The P value is 41.1% which is much higher than the 5% Significance Level. This suggests that the sample is consistent with the H0 (original control hypothesis). This means that the new design did not perform better than the original control designs. 

The confidence interval for the treatment group is 47.3% to 52.3%. An imapct effect from 50% to 55% was desired. Therefore some improvement was observed with the home page design but it did not reach the required 55%.